In [99]:
import pandas as pd
import spacy
from time import time

Dataset -- amazon book reviews
https://www.kaggle.com/datasets/meetnagadia/amazon-kindle-book-review-for-sentiment-analysis

Подивимося на дані, видалимо лишні стовпчики, проведемо препроцессинг (токенізація + видалимо стоп-слова, пунктуацію тощо)

In [100]:
texts = pd.read_csv("preprocessed_kindle_review.csv")
texts.head()

,Unnamed: 0,rating,reviewText,summary
0,0,5,This book was the very first bookmobile book I...,50 + years ago...
1,1,1,"When I read the description for this book, I c...",Boring! Boring! Boring!
2,2,5,I just had to edit this review. This book is a...,Wiggleliscious/new toy ready/!!
3,3,5,I don't normally buy 'mystery' novels because ...,Very good read.
4,4,5,"This isn't the kind of book I normally read, a...",Great Story!


In [101]:
texts.drop(columns=['Unnamed: 0', 'summary', 'rating'], inplace=True)

In [102]:
nlp = spacy.load("en_core_web_sm")
def preprocess_doc(doc):
    processed_text = " ".join([token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha])
    return processed_text

In [103]:
t = time()

texts["preprocessed_text"] = [preprocess_doc(doc) for doc in nlp.pipe(texts["reviewText"], n_process=-1)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))
texts.head()

Time to clean up everything: 1.52 mins


,reviewText,preprocessed_text
0,This book was the very first bookmobile book I...,book bookmobile book buy school book club love...
1,"When I read the description for this book, I c...",read description book wait read download kindl...
2,I just had to edit this review. This book is a...,edit review book believe get right update writ...
3,I don't normally buy 'mystery' novels because ...,normally buy mystery novel like time decide ch...
4,"This isn't the kind of book I normally read, a...",kind book normally read try limit certain genr...


In [104]:
texts = texts.dropna().drop_duplicates()
texts.shape

(12000, 2)

### 1. Використати алгоритми/моделі tf-idf, BoW, Word2Vec, Doc2Vec для feature extraction.

TF-IDF

In [105]:
from gensim import corpora
from gensim.models import TfidfModel

# Create a list of tokenized documents
tokenized_reviews = [doc.split() for doc in texts["preprocessed_text"]]

# Create a dictionary from the tokenized documents
dictionary = corpora.Dictionary(tokenized_reviews)

# Create a corpus from the tokenized documents
corpus = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

# Create the TF-IDF model
tfidf_model = TfidfModel(corpus)

# Compute the TF-IDF scores
tfidf_scores = tfidf_model[corpus]

In [106]:
for i, doc_scores in enumerate(tfidf_scores[:3]):
    print(f"Document {i+1}:")
    for word_id, score in doc_scores:
        word = dictionary[word_id]
        print(f"\tWord: {word}, TF-IDF Score: {score}")

Document 1:
	Word: bet, TF-IDF Score: 0.25635241285087756
	Word: book, TF-IDF Score: 0.08556797458511392
	Word: bookmobile, TF-IDF Score: 0.3996158090740527
	Word: buy, TF-IDF Score: 0.23021841678644553
	Word: club, TF-IDF Score: 0.20454592340564887
	Word: dollar, TF-IDF Score: 0.23069762301281474
	Word: donut, TF-IDF Score: 0.37012549275398793
	Word: forward, TF-IDF Score: 0.1403616346693509
	Word: grade, TF-IDF Score: 0.2471530854305076
	Word: look, TF-IDF Score: 0.09044476556124408
	Word: love, TF-IDF Score: 0.11236331629378038
	Word: memory, TF-IDF Score: 0.41446780691278184
	Word: relive, TF-IDF Score: 0.37012549275398793
	Word: school, TF-IDF Score: 0.16319759679285945
	Word: serve, TF-IDF Score: 0.21060084294839582
	Word: story, TF-IDF Score: 0.02933460133533035
Document 2:
	Word: book, TF-IDF Score: 0.05355708720723589
	Word: love, TF-IDF Score: 0.05274624027404937
	Word: story, TF-IDF Score: 0.009180279243042308
	Word: aaron, TF-IDF Score: 0.5521617339171142
	Word: accommodate

BoW

In [107]:
from collections import Counter

In [108]:
sorted(tokenized_reviews[0])

['bet',
 'book',
 'book',
 'book',
 'book',
 'bookmobile',
 'buy',
 'buy',
 'club',
 'dollar',
 'donut',
 'forward',
 'grade',
 'look',
 'love',
 'love',
 'memory',
 'memory',
 'relive',
 'school',
 'serve',
 'story']

In [109]:
dictionary.doc2bow(tokenized_reviews[0])

[(0, 1),
 (1, 4),
 (2, 1),
 (3, 2),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 2),
 (11, 2),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1)]

In [110]:
bag_of_words = []
unique_words = set()

for tokenized_text in tokenized_reviews:
    frequency = Counter(tokenized_text)
    bag_of_words.append(frequency)
    unique_words.update(tokenized_text)

print(bag_of_words[0])
print(bag_of_words[1])
print(len(unique_words))

Counter({'book': 4, 'buy': 2, 'love': 2, 'memory': 2, 'bookmobile': 1, 'school': 1, 'club': 1, 'story': 1, 'bet': 1, 'dollar': 1, 'donut': 1, 'serve': 1, 'grade': 1, 'look': 1, 'forward': 1, 'relive': 1})
Counter({'book': 8, 'read': 7, 'aaron': 5, 'passion': 4, 'father': 4, 'finally': 4, 'love': 3, 'jada': 3, 'tell': 3, 'kindle': 2, 'find': 2, 'stop': 2, 'slow': 2, 'blah': 2, 'child': 2, 'son': 2, 'know': 2, 'good': 2, 'let': 2, 'scene': 2, 'sweet': 2, 'believe': 2, 'description': 1, 'wait': 1, 'download': 1, 'extremely': 1, 'hard': 1, 'honest': 1, 'halfway': 1, 'begin': 1, 'remain': 1, 'uninteresting': 1, 'lack': 1, 'make': 1, 'life': 1, 'interesting': 1, 'character': 1, 'story': 1, 'perfect': 1, 'everybody': 1, 'understanding': 1, 'accommodate': 1, 'bit': 1, 'drama': 1, 'grandmother': 1, 'example': 1, 'mean': 1, 'half': 1, 'way': 1, 'year': 1, 'ago': 1, 'mind': 1, 'keep': 1, 'contact': 1, 'month': 1, 'leave': 1, 'mention': 1, 'pregnant': 1, 'overwhelmed': 1, 'emotion': 1, 'misty': 1,

Word2Vec

In [111]:
import multiprocessing
from gensim.models import Word2Vec

In [112]:
cores = multiprocessing.cpu_count()

In [113]:
w2v_model = Word2Vec(min_count=1,
                     window=1,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=cores-1)

In [114]:
t = time()
w2v_model.build_vocab(tokenized_reviews, progress_per=1)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.01 mins


In [115]:
w2v_model.wv.most_similar("science")

[('groovy', 0.3726963400840759),
 ('muffin', 0.3443751335144043),
 ('jet', 0.34161823987960815),
 ('ths', 0.34132227301597595),
 ('perfectionist', 0.3407387137413025),
 ('kelly', 0.33902761340141296),
 ('stiletto', 0.33505386114120483),
 ('negate', 0.33131006360054016),
 ('actuality', 0.3292902112007141),
 ('management', 0.320794939994812)]

In [116]:
#train model
t = time()
w2v_model.train(tokenized_reviews, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.14 mins


In [117]:
w2v_model.wv.most_similar("science")

[('asimov', 0.8096764087677002),
 ('analog', 0.7690597772598267),
 ('speculative', 0.7681789398193359),
 ('pulp', 0.7666961550712585),
 ('technology', 0.7505962252616882),
 ('flash', 0.7325021028518677),
 ('fiction', 0.718043327331543),
 ('trend', 0.706889808177948),
 ('advanced', 0.6995603442192078),
 ('releax', 0.6983894109725952)]

Doc2Vec

In [118]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [119]:
documents = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(tokenized_reviews)]
d2v_model = Doc2Vec(documents, vector_size=100, min_count=1, epochs=30)

In [120]:
d2v_model.dv[0]

array([-0.11095981, -0.06486581, -0.11347137, -0.32091355, -0.06308714,
        0.05321413, -0.09706122, -0.2213369 ,  0.02773999, -0.37636384,
       -0.1746189 ,  0.12287342,  0.01779163, -0.09120695,  0.27890617,
       -0.20330189, -0.13827841, -0.20036004, -0.11344602, -0.13184811,
       -0.2670707 ,  0.26829854, -0.05612657, -0.23672609,  0.2820123 ,
        0.12936348, -0.08084273, -0.17784114, -0.14439514, -0.05022502,
       -0.1377238 ,  0.11497473,  0.17679104, -0.20030618, -0.02092261,
        0.17777343, -0.07151854,  0.08187978, -0.00783767, -0.21574773,
       -0.12403399, -0.0944458 ,  0.10449786, -0.44210273,  0.02995184,
        0.00679747, -0.37527898, -0.05209398, -0.25792846, -0.16684988,
       -0.07457037, -0.06058825,  0.08761381, -0.28987744,  0.08523968,
        0.13205802,  0.04912262, -0.2227792 , -0.3765109 ,  0.11402324,
       -0.10014623,  0.2563076 , -0.17024802, -0.03615481,  0.15521415,
        0.12434776, -0.10478158,  0.04515597, -0.03997979, -0.15

In [121]:
new_review = "I really loved this book. It was so captivating I've read entire book in a day!"
new_review_clean = preprocess_doc(nlp(new_review))
print(new_review_clean)

new_vector = d2v_model.infer_vector(new_review_clean.split(), epochs=100)
d2v_model.dv.most_similar([new_vector])

love book captivate read entire book day


[(1739, 0.6759413480758667),
 (8821, 0.5977339148521423),
 (10612, 0.5970027446746826),
 (3823, 0.5897597670555115),
 (313, 0.5877304673194885),
 (795, 0.5861930251121521),
 (10729, 0.5811052322387695),
 (2242, 0.572374165058136),
 (9777, 0.5704280138015747),
 (1041, 0.5702077746391296)]

In [122]:
documents[11238]

TaggedDocument(words=['love', 'story', 'love', 'preview', 'book', 'wait', 'read'], tags=[11238])

__________________________________________________________________________________________________________________________________________________________

### 2. Summarization

In [123]:
num_words = 15

tf-idf

In [124]:
summarized_texts_tfidf = []
for doc_id, doc_scores in enumerate(tfidf_scores):
    top_n = sorted(doc_scores, key=lambda x: -x[1])[:num_words]
    summarized_texts_tfidf.append(' '.join([dictionary[i] for i, _ in top_n]))
texts['summarized_tf-idf'] = summarized_texts_tfidf

In [125]:
texts

,reviewText,preprocessed_text,summarized_tf-idf
0,This book was the very first bookmobile book I...,book bookmobile book buy school book club love...,memory bookmobile donut relive bet grade dolla...
1,"When I read the description for this book, I c...",read description book wait read download kindl...,aaron jada passion father finally blah barf so...
2,I just had to edit this review. This book is a...,edit review book believe get right update writ...,bechett hmmmm ha barrett luv scottie battery c...
3,I don't normally buy 'mystery' novels because ...,normally buy mystery novel like time decide ch...,mystery engross buy novel exception normally e...
4,"This isn't the kind of book I normally read, a...",kind book normally read try limit certain genr...,johnston ty epic tedious limit alive trilogy p...
...,...,...,...
11995,Had to read certain passages twice--typos. Wi...,read certain passage twice typo wish build rel...,passage twice typo certain chemistry build wis...
11996,Not what i expected. yet a very interesting bo...,expect interesting book usually read book era ...,era usually interest expect interesting go boo...
11997,Dragon Knights is a world where Knights ride d...,dragon knights world knights ride dragon slay ...,dragon knights backgroung firedrake gryphons h...
11998,"Since this story is very short, it's hard to s...",story short hard give away plot story war batt...,seamlessly effective slip battle reality war s...


Word2Vec

In [126]:
import numpy as np

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [127]:
num_words = 15
summarized_texts_w2v = []
for doc in tokenized_reviews:
    doc_words_and_embeddings = [(word, w2v_model.wv[word]) for word in doc if word in w2v_model.wv]
    doc_embeddings_only = [x[1] for x in doc_words_and_embeddings]
    doc_avg_embedding = sum(doc_embeddings_only) / len(doc_embeddings_only)
    similarity_scores = [(word, cosine_similarity(word_embedding, doc_avg_embedding)) for (word, word_embedding) in doc_words_and_embeddings]
    summary = ' '.join([word for word, _ in sorted(similarity_scores, key=lambda x: x[1])[:num_words]])
    summarized_texts_w2v.append(summary)
texts['summarized_w2v'] = summarized_texts_w2v

In [128]:
texts

,reviewText,preprocessed_text,summarized_tf-idf,summarized_w2v
0,This book was the very first bookmobile book I...,book bookmobile book buy school book club love...,memory bookmobile donut relive bet grade dolla...,serve club school forward memory memory look l...
1,"When I read the description for this book, I c...",read description book wait read download kindl...,aaron jada passion father finally blah barf so...,eat torture theme tooth blah blah kindle kindl...
2,I just had to edit this review. This book is a...,edit review book believe get right update writ...,bechett hmmmm ha barrett luv scottie battery c...,king max wall study war match new heat large m...
3,I don't normally buy 'mystery' novels because ...,normally buy mystery novel like time decide ch...,mystery engross buy novel exception normally e...,decide chance time mystery mystery engage cons...
4,"This isn't the kind of book I normally read, a...",kind book normally read try limit certain genr...,johnston ty epic tedious limit alive trilogy p...,hold word alive trilogy try fantasy limit para...
...,...,...,...,...
11995,Had to read certain passages twice--typos. Wi...,read certain passage twice typo wish build rel...,passage twice typo certain chemistry build wis...,twice certain read typo build relationship che...
11996,Not what i expected. yet a very interesting bo...,expect interesting book usually read book era ...,era usually interest expect interesting go boo...,era interest go usually interesting expect rea...
11997,Dragon Knights is a world where Knights ride d...,dragon knights world knights ride dragon slay ...,dragon knights backgroung firedrake gryphons h...,advance change dream ride world novella read r...
11998,"Since this story is very short, it's hard to s...",story short hard give away plot story war batt...,seamlessly effective slip battle reality war s...,war battle away reality fantasy give writing s...


### 3. Named Entity Recognition

In [138]:
summary_tfidf_ner = []
summary_w2v_ner = []
for summary in texts['summarized_tf-idf']:
    doc = nlp(summary)
    summary_tfidf_ner.append(doc.ents)
texts['ner_tfidf'] = summary_tfidf_ner

for summary in texts['summarized_w2v']:
    doc = nlp(summary)
    summary_w2v_ner.append(doc.ents)
texts['ner_w2v'] = summary_w2v_ner

In [139]:
texts

,reviewText,preprocessed_text,summarized_tf-idf,summarized_w2v,ner_tfidf,ner_w2v
0,This book was the very first bookmobile book I...,book bookmobile book buy school book club love...,memory bookmobile donut relive bet grade dolla...,serve club school forward memory memory look l...,(),()
1,"When I read the description for this book, I c...",read description book wait read download kindl...,aaron jada passion father finally blah barf so...,eat torture theme tooth blah blah kindle kindl...,"((aaron, jada), (harvard, accommodate))","((blah, blah, kindle, kindle),)"
2,I just had to edit this review. This book is a...,edit review book believe get right update writ...,bechett hmmmm ha barrett luv scottie battery c...,king max wall study war match new heat large m...,(),"((max, wall),)"
3,I don't normally buy 'mystery' novels because ...,normally buy mystery novel like time decide ch...,mystery engross buy novel exception normally e...,decide chance time mystery mystery engage cons...,(),()
4,"This isn't the kind of book I normally read, a...",kind book normally read try limit certain genr...,johnston ty epic tedious limit alive trilogy p...,hold word alive trilogy try fantasy limit para...,"((johnston, ty),)",()
...,...,...,...,...,...,...
11995,Had to read certain passages twice--typos. Wi...,read certain passage twice typo wish build rel...,passage twice typo certain chemistry build wis...,twice certain read typo build relationship che...,(),()
11996,Not what i expected. yet a very interesting bo...,expect interesting book usually read book era ...,era usually interest expect interesting go boo...,era interest go usually interesting expect rea...,(),()
11997,Dragon Knights is a world where Knights ride d...,dragon knights world knights ride dragon slay ...,dragon knights backgroung firedrake gryphons h...,advance change dream ride world novella read r...,"((dragon, knights, backgroung, firedrake, gryp...","((knight, dragon),)"
11998,"Since this story is very short, it's hard to s...",story short hard give away plot story war batt...,seamlessly effective slip battle reality war s...,war battle away reality fantasy give writing s...,(),()


In [146]:
#cross one out
print(w2v_model.wv.doesnt_match(['book', 'piano', 'read']))
print(w2v_model.wv.doesnt_match(['book', 'science', 'water']))

piano
water


In [151]:
w2v_model.wv.most_similar(['student', 'university'])

[('grad', 0.9258071184158325),
 ('graduate', 0.9110099077224731),
 ('professor', 0.9031205177307129),
 ('freshman', 0.8927655816078186),
 ('math', 0.8722860813140869),
 ('junior', 0.8531510233879089),
 ('seminary', 0.8518487811088562),
 ('realizes', 0.8482683897018433),
 ('swordmaster', 0.8427854776382446),
 ('semester', 0.8362836241722107)]

In [155]:
from rouge import Rouge
rouge = Rouge()

tfidf_metrics = []
w2v_metrics = []

for reference, tfidf_sum, w2v_sum in zip(texts['preprocessed_text'], texts['summarized_tf-idf'], texts['summarized_w2v']):
    tfidf_metrics.append(rouge.get_scores(reference, tfidf_sum))
    w2v_metrics.append(rouge.get_scores(reference, w2v_sum))

In [161]:
tfidf_metrics[123]

[[{'rouge-1': {'r': 1.0, 'p': 0.9375, 'f': 0.967741930489074},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r': 0.3333333333333333,
    'p': 0.3125,
    'f': 0.3225806401664933}}]]

In [163]:
w2v_metrics[123]

[{'rouge-1': {'r': 1.0, 'p': 0.2549019607843137, 'f': 0.40624999676269535},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.46153846153846156,
   'p': 0.11764705882352941,
   'f': 0.18749999676269535}}]